In [ ]:
#hide
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

%load_ext autoreload
%load_ext line_profiler
%load_ext cython
%autoreload 2 

In [ ]:
#exporti
import os
from pathlib import Path
from typing import *
from collections import defaultdict, namedtuple
from fastcore.basics import partialler, listify
from fastcore.utils import parallel
from fastcore.foundation import L
from rfpye.constants import BYTES_HEADER, ENDMARKER, KEY_ATTRS
from rfpye.blocks import MAIN_BLOCKS
from rfpye.utils import get_files, getattrs, bin2int, bin2str
from rfpye.parser import *
from rfpye.cyparser import cy_extract_compressed
from loguru import logger
import pandas as pd
import numpy as np

In [ ]:
files = get_files(r'D:\OneDrive - ANATEL\Sensores', extensions=['.bin'])
file = files.shuffle()[0]

In [ ]:
file = Path('D:/OneDrive - ANATEL/Sensores/rfeye002295/rfeye002295_210622_T220240.bin')

In [ ]:
blocks

{'filename': 'rfeye002295_210622_T220240.bin',
 'file_version': 23,
 'string': 'CRFS DATA FILE V023',
 'hostname': 'rfeye002295',
 'method': 'ScriptRFeye2021_v2.cfg',
 'unit_info': 'Stationary',
 'file_number': 0,
 'identifier': 'INFO',
 'gps': GPS Data - Median of Coordinates: -3.02318:-60.05499 Altitude: 105.00 #Satellites: 12.0,
 'spectrum': (#20) [SpecData(type=67, thread_id=300, description='PMEC 2021 (Faixa 1 de 10).', start_mega=105.0, stop_mega=140.0, dtype='dBm', ndata=3584, bw=18457, processing='peak', antuid=0),SpecData(type=67, thread_id=310, description='PMEC 2021 (Faixa 2 de 10).', start_mega=155.0, stop_mega=165.0, dtype='dBm', ndata=1024, bw=18457, processing='peak', antuid=0),SpecData(type=67, thread_id=100, description='PRD 2021 (Faixa principal 1 de 4).', start_mega=50.0, stop_mega=90.0, dtype='dBμV/m', ndata=1024, bw=73828, processing='peak', antuid=0),SpecData(type=67, thread_id=110, description='PRD 2021 (Faixa principal 2 de 4).', start_mega=70.0, stop_mega=110.0

In [ ]:
%lprun -f classify_blocks blocks = parse_bin(file)

Timer unit: 3.52617e-07 s

Total time: 7.30025 s
File: C:\Users\rsilva\Code\rfpye\rfpye\parser.py
Function: classify_blocks at line 231

Line #      Hits         Time  Per Hit   % Time  Line Contents
   231                                           def classify_blocks(byte_blocks: Iterable) -> dict:
   232                                               """Receives an iterable with binary blocks and returns a dict with the metadata from file, the gps class and a list with the different spectrum classes"""
   233         1         12.0     12.0      0.0      meta = {}
   234         1          2.0      2.0      0.0      fluxos = {}
   235         1         67.0     67.0      0.0      gps = CrfsGPS()
   236     45103     106188.0      2.4      0.5      for attrs, block in byte_blocks:
   237                                                   #block = create_block(byte_block)
   238     45102      58087.0      1.3      0.3          if not block:
   239      3970       4129.0      1.0      0.

In [ ]:
%%cython --annotate
cimport cython

ctypedef object CrfsGPS

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef dict classify_blocks(list byte_blocks):
    """Receives an iterable with binary blocks and returns a dict with the metadata from file, the gps class and a list with the different spectrum classes"""
    
    cdef dict meta = {}
    cdef dict spec = {}
    cdef object gps
    cdef list gps_attrs
    gps = CrfsGPS()
    gps_attrs = [f'_{k}' for k in BLOCK_ATTRS.get(40, [])]
    for byte_block in byte_blocks:
        block = create_block(byte_block)
        if not block: continue 
        if block.type  == 40:
            for k in BLOCK_ATTRS.get(40, []):
                getattr(gps, f'_{k}').append(getattr(block, k))
            continue
        attrs=KEY_ATTRS.get(block.type, [])
        values = getattrs(block, attrs=attrs)
        if block.type in SPECTRAL_BLOCKS:
            if values not in fluxos:
                attrs = [a for a in attrs] + ['thresh', 'minimum']
                metadata = namedtuple('metadata', attrs)
                attrs = list(values) + [block.thresh, block.minimum]
                spec[values] = CrfsSpectrum(metadata(*attrs))
            time = getattr(block, 'wallclock_datetime')
            attr = 'raw_data' if block.type in (64, 68) else 'levels'
            data = getattr(block, attr)
            spec[values].append(time, data)
        else:
            meta.update(dict(zip(attrs, values)))
    meta['gps'] = gps
    meta['spectrum'] = list(spec.values()) 
    return meta